In [1]:

import pygame
import sys as sys

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
import pygame, sys

# Initialize Pygame
pygame.init()

# Screen dimensions and window setup
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Twin Collision Game")

# Colors and Fonts
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 50)

# Game states
MENU = 0
GAME = 1
OPTIONS = 2
current_state = MENU

# Menu options
menu_options = ["Start Game", "Options", "Exit"]
selected_option = 0

# ---------------------------
# Helper function to load images
def load_images(paths, scale=None):
    images = []
    for path in paths:
        img = pygame.image.load(path).convert_alpha()
        if scale:
            img = pygame.transform.scale(img, scale)
        images.append(img)
    return images

# ---------------------------
# Player class with both movement and idle animations
class Player(pygame.sprite.Sprite):
    def __init__(self, move_animations, idle_animations, speed, pos):
        """
        move_animations: dict with keys "left", "right", "up", "down" for movement animations.
        idle_animations: dict with keys "left", "right", "up", "down" for idle animations.
        speed: movement speed in pixels per second.
        pos: a pygame.Rect for starting position and size.
        """
        super().__init__()
        self.move_animations = move_animations
        self.idle_animations = idle_animations
        self.direction = "down"  # default direction
        self.index = 0
        # Start with idle image
        self.image = self.idle_animations[self.direction][0]
        self.rect = pos
        self.speed = speed
        self.last_update = pygame.time.get_ticks()
        self.animation_delay = 150  # milliseconds between frames

    def update_animation(self, moving):
        if moving:
            now = pygame.time.get_ticks()
            if now - self.last_update > self.animation_delay:
                self.last_update = now
                self.index = (self.index + 1) % len(self.move_animations[self.direction])
                self.image = self.move_animations[self.direction][self.index]
        else:
            self.index = 0
            self.image = self.idle_animations[self.direction][0]

    def move(self, dx, dy):
        self.rect.x += dx
        self.rect.y += dy

# ---------------------------
# Load animations
# NOTE: File name swap - player1 uses the "green_brother" images (for the purple character),
# while player2 uses the "twin2" images (for the green character).

# Player1 (Purple Character) movement animations:
green_move = {
    "left": load_images(["green_brother_left_idle.png", "green_brother_left_walking.png"], scale=(80, 80)),
    "right": load_images(["green_brother_right_idle.png", "green_brother_right_walking.png"], scale=(80, 80)),
    "up": load_images(["green_brother_Rleg_up.png", "green_brother_Lleg_up.png"], scale=(80, 80)),
    "down": load_images(["green_brother_Lleg_down.png", "green_brother_Rleg_down.png"], scale=(80, 80))
}
# Player1 (Purple Character) idle animations:
green_idle = {
    "left": load_images(["green_brother_front_idle.png"], scale=(80, 80)),
    "right": load_images(["green_brother_front_idle.png"], scale=(80, 80)),
    "up": load_images(["green_brother_front_idle.png"], scale=(80, 80)),
    "down": load_images(["green_brother_front_idle.png"], scale=(80, 80))
}

# Player2 (Green Character) movement animations:
purple_move = {
    "left": load_images(["purple_bro_left_idle.png", "purple_bro_left_walking.png"], scale= (80, 80)),
    "right": load_images(["purple_brother_right_idle.png", "purple_brother_right_walking.png"], scale=(80, 80)),
    "up": load_images(["purple_brother_Rleg_up.png", "purple_brother_Lleg_up.png"], scale=(80, 80)),
    "down": load_images(["purple_brother_Lleg_down.png", "purple_brother_Rleg_down.png"], scale=(80, 80))
}
# Player2 (Green Character) idle animations:
purple_idle = {
    "left": load_images(["purple_bro_front_idle.png"], scale=(80, 80)),
    "right": load_images(["purple_bro_front_idle.png"], scale=(80, 80)),
    "up": load_images(["purple_bro_front_idle.png"], scale=(80, 80)),
    "down": load_images(["purple_bro_front_idle.png"], scale=(80, 80))
}

# Create players with starting positions.
player1 = Player(purple_move, purple_idle, speed=200, pos=pygame.Rect(100, 100, 50, 50))
player2 = Player(green_move, green_idle, speed=200, pos=pygame.Rect(400, 300, 50, 50))

# ---------------------------
# Control schemes for two players
controls_player1 = {
    "up": pygame.K_w,
    "down": pygame.K_s,
    "left": pygame.K_a,
    "right": pygame.K_d,
}

controls_player2 = {
    "up": pygame.K_UP,
    "down": pygame.K_DOWN,
    "left": pygame.K_LEFT,
    "right": pygame.K_RIGHT,
}

# ---------------------------
# Movement and collision handling (using first code's logic)
def move_players_with_collision(player1, player2, keys, dt, controls1, controls2):
    # Compute movement deltas for player1
    normal_move1 = player1.speed * dt
    dx1 = dy1 = 0
    if keys[controls1["up"]]:
        dy1 -= normal_move1
        player1.direction = "up"
    if keys[controls1["down"]]:
        dy1 += normal_move1
        player1.direction = "down"
    if keys[controls1["left"]]:
        dx1 -= normal_move1
        player1.direction = "left"
    if keys[controls1["right"]]:
        dx1 += normal_move1
        player1.direction = "right"
    if keys[pygame.K_LSHIFT]:
        dx1 *= 2
        dy1 *= 2

    # Compute movement deltas for player2
    normal_move2 = player2.speed * dt
    dx2 = dy2 = 0
    if keys[controls2["up"]]:
        dy2 -= normal_move2
        player2.direction = "up"
    if keys[controls2["down"]]:
        dy2 += normal_move2
        player2.direction = "down"
    if keys[controls2["left"]]:
        dx2 -= normal_move2
        player2.direction = "left"
    if keys[controls2["right"]]:
        dx2 += normal_move2
        player2.direction = "right"
    if keys[pygame.K_RSHIFT]:  # Using right shift for second player if desired
        dx2 *= 2
        dy2 *= 2

    # Horizontal movement and collision
    old_x1, old_x2 = player1.rect.x, player2.rect.x
    player1.rect.x += dx1
    player2.rect.x += dx2
    if player1.rect.colliderect(player2.rect):
        if abs(dx1) > abs(dx2):
            player1.rect.x = old_x1
        elif abs(dx1) < abs(dx2):
            player2.rect.x = old_x2
        else:
            player1.rect.x = old_x1
            player2.rect.x = old_x2

    # Vertical movement and collision
    old_y1, old_y2 = player1.rect.y, player2.rect.y
    player1.rect.y += dy1
    player2.rect.y += dy2
    if player1.rect.colliderect(player2.rect):
        if abs(dy1) > abs(dy2):
            player1.rect.y = old_y1
        elif abs(dy1) < abs(dy2):
            player2.rect.y = old_y2
        else:
            player1.rect.y = old_y1
            player2.rect.y = old_y2

    # Update animations based on movement
    player1.update_animation(dx1 != 0 or dy1 != 0)
    player2.update_animation(dx2 != 0 or dy2 != 0)

# ---------------------------
# Drawing functions for different game states
def draw_menu():
    screen.fill(WHITE)
    title = font.render("Twin Collision Game", True, BLACK)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 100))
    for i, option in enumerate(menu_options):
        color = BLACK if i == selected_option else GRAY
        text = small_font.render(option, True, color)
        screen.blit(text, (WIDTH // 2 - text.get_width() // 2, 250 + i * 70))

def draw_game():
    screen.fill(WHITE)
    screen.blit(player1.image, player1.rect)
    screen.blit(player2.image, player2.rect)

def draw_options():
    screen.fill(WHITE)
    title = font.render("Options", True, BLACK)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 100))
    text = small_font.render("Options screen.", True, BLACK)
    screen.blit(text, (WIDTH // 2 - text.get_width() // 2, 300))

# ---------------------------
# Main game loop with menu and options
clock = pygame.time.Clock()
running = True
while running:
    dt = clock.tick(60) / 1000  # Delta time in seconds

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Menu navigation
        if current_state == MENU:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN:
                    selected_option = (selected_option + 1) % len(menu_options)
                if event.key == pygame.K_UP:
                    selected_option = (selected_option - 1) % len(menu_options)
                if event.key == pygame.K_RETURN:
                    if selected_option == 0:  # Start Game
                        current_state = GAME
                    elif selected_option == 1:  # Options
                        current_state = OPTIONS
                    elif selected_option == 2:  # Exit
                        running = False

        # Options screen: press Escape to go back to menu
        elif current_state == OPTIONS:
            if event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE:
                current_state = MENU

    # ---------------------------
    # Update game state
    if current_state == GAME:
        keys = pygame.key.get_pressed()
        move_players_with_collision(player1, player2, keys, dt, controls_player1, controls_player2)

    # ---------------------------
    # Drawing based on state
    if current_state == MENU:
        draw_menu()
    elif current_state == GAME:
        draw_game()
    elif current_state == OPTIONS:
        draw_options()

    pygame.display.flip()

pygame.quit()
sys.exit()


SystemExit: 